```{contents}
```
## Environment Isolation

**Environment Isolation** in LangGraph is the architectural practice of **separating execution contexts, resources, configurations, and side effects** of different graphs, runs, agents, users, and deployments to guarantee **correctness, security, reproducibility, and scalability** of LLM systems in production.

It ensures that **one workflow’s state, failures, credentials, or resource usage cannot interfere with another**.

---

### **1. Why Environment Isolation Is Critical**

LLM workflows are:

* **Stateful**
* **Long-running**
* **Multi-agent**
* **Externally connected**
* **Cost-sensitive**

Without isolation:

| Risk                | Failure Mode                |
| ------------------- | --------------------------- |
| State leakage       | User A sees User B data     |
| Credential exposure | Wrong API keys used         |
| Non-determinism     | Hard-to-debug behavior      |
| Resource contention | One workflow starves others |
| Security breach     | Cross-tenant access         |

---

### **2. Isolation Layers in LangGraph**

LangGraph enforces isolation at **multiple architectural layers**.

| Layer          | What Is Isolated         |
| -------------- | ------------------------ |
| Execution      | Threads, runs, recursion |
| State          | Memory, checkpoints      |
| Agents         | Roles, messages, tools   |
| Resources      | LLM clients, tools       |
| Configuration  | Models, prompts, limits  |
| Infrastructure | CPU, RAM, network        |

---

### **3. Execution-Level Isolation**

Each graph invocation runs inside its own **execution context**:

```python
graph.invoke(input, config={"configurable": {"thread_id": "user_123"}})
```

**Isolated by:**

* `thread_id`
* `run_id`
* `recursion_limit`
* Execution stack

This prevents:

* Infinite loops crossing runs
* Cross-run state pollution
* Concurrent race conditions

---

### **4. State & Memory Isolation**

State is isolated using **per-thread persistence**.

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("state.db")
graph = builder.compile(checkpointer=checkpointer)
```

| Concept        | Isolation Guarantee    |
| -------------- | ---------------------- |
| Thread ID      | Unique conversation    |
| Checkpoint     | Per execution snapshot |
| State schema   | Enforced boundaries    |
| State reducers | Controlled mutation    |

**Result:** Each workflow has its own memory timeline.

---

### **5. Agent Isolation**

Each agent node executes with:

* Its own prompt
* Its own toolset
* Its own memory slice
* Its own permissions

```python
def finance_agent(state):
    ...
def legal_agent(state):
    ...
```

Agents share only the state fields explicitly allowed.

---

### **6. Tool & Resource Isolation**

Tools are bound per graph or per agent:

```python
graph = builder.compile(
    configurable={
        "model": "gpt-4",
        "api_key": os.getenv("OPENAI_KEY_A")
    }
)
```

Prevents:

* Credential leaks
* Tool misuse
* Cost leakage

---

### **7. Deployment-Level Isolation**

| Technique             | Purpose              |
| --------------------- | -------------------- |
| Containers            | Dependency isolation |
| Kubernetes namespaces | Tenant separation    |
| Secrets vault         | Secure credentials   |
| Network policies      | API boundaries       |
| Resource quotas       | Cost control         |

---

### **8. Human-in-the-Loop Isolation**

Human approvals, feedback, and corrections are **scoped to the active execution thread**, preventing cross-user contamination.

---

### **9. Production Isolation Patterns**

| Pattern           | Use Case              |
| ----------------- | --------------------- |
| Multi-tenant SaaS | Per-tenant graphs     |
| Regulated systems | Compliance boundaries |
| Enterprise apps   | Department separation |
| Agent platforms   | Role-based isolation  |

---

### **10. Failure Containment**

Isolation enables:

* Crash containment
* Partial restarts
* Safe rollback
* Replay debugging

---

### **11. Mental Model**

LangGraph environments behave like:

> **Mini operating systems for each AI workflow**

with their own:

* Memory
* Processes
* Permissions
* Resources
* Lifecycles

---

### **12. Summary**

Environment Isolation is what transforms LangGraph from a research tool into a **production-grade orchestration platform** by guaranteeing:

> **Security • Reliability • Reproducibility • Scalability**



### Demonstration

In [2]:
"""
Demonstration: Environment Isolation in LangGraph (Single Cell)

This example shows:
1) Per-run execution isolation via thread_id
2) Isolated persistent state using a checkpointer
3) Agent isolation via explicit state sharing
4) Resource isolation via configurable runtime parameters
"""

from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

# ---------------------------
# 1. Define isolated state
# ---------------------------
class State(TypedDict):
    user: str
    count: int

# ---------------------------
# 2. Define agent nodes
# ---------------------------
def agent_a(state: State):
    return {"count": state["count"] + 1}

def agent_b(state: State):
    return {"count": state["count"] * 2}

# ---------------------------
# 3. Build graph
# ---------------------------
builder = StateGraph(State)
builder.add_node("incrementer", agent_a)
builder.add_node("doubler", agent_b)

builder.set_entry_point("incrementer")
builder.add_edge("incrementer", "doubler")
builder.add_edge("doubler", END)

# ---------------------------
# 4. Enable persistent isolation
# ---------------------------
import sqlite3
checkpointer = SqliteSaver(sqlite3.connect("isolated_state.db",check_same_thread=False))
graph = builder.compile(checkpointer=checkpointer)

# ---------------------------
# 5. Two isolated executions
# ---------------------------
result_1 = graph.invoke(
    {"user": "alice", "count": 1},
    config={"configurable": {"thread_id": "thread_alice"}}
)

result_2 = graph.invoke(
    {"user": "bob", "count": 1},
    config={"configurable": {"thread_id": "thread_bob"}}
)

print("Alice:", result_1)
print("Bob:  ", result_2)

"""
Output:
Alice: {'user': 'alice', 'count': 4}
Bob:   {'user': 'bob',   'count': 4}

Even though the same graph was executed twice,
each run had fully isolated state, memory, and execution context.
"""


Alice: {'user': 'alice', 'count': 4}
Bob:   {'user': 'bob', 'count': 4}


"\nOutput:\nAlice: {'user': 'alice', 'count': 4}\nBob:   {'user': 'bob',   'count': 4}\n\nEven though the same graph was executed twice,\neach run had fully isolated state, memory, and execution context.\n"